<a href="https://colab.research.google.com/github/RafifAlzayat/thecoolteam-/blob/ava-branch/Capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InR

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Yelp_NLP").getOrCreate()

In [3]:
import json
import pandas as pd
import numpy as np
import requests
from pyspark.sql import functions as F
from urllib.request import Request, urlopen
from pyspark import SparkContext

In [ ]:
#https://data.cdc.gov/api/views/n8mc-b4w4/rows.json?accessType=DOWNLOAD
#https://rafifcoviddata.s3.amazonaws.com/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv

In [4]:
from pyspark import SparkFiles
url ="https://rafifcoviddata.s3.amazonaws.com/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"
spark.sparkContext.addFile(url)
covid_data_df = spark.read.csv(SparkFiles.get("COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"), sep=",", header=True, inferSchema=True)

In [5]:
covid_data_df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|            process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-06|       OH|             39|   BELMONT|           39013|            NA|    NA|     NA|       NA|                              0|               null|           

In [6]:
#dropped na values
covid_data_df_dropped = covid_data_df.dropna(how = 'all')

In [7]:
covid_data_df_dropped.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|            process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-06|       OH|             39|   BELMONT|           39013|            NA|    NA|     NA|       NA|                              0|               null|           

In [8]:
covid_data_df_dropped

DataFrame[case_month: string, res_state: string, state_fips_code: string, res_county: string, county_fips_code: string, age_group: string, sex: string, race: string, ethnicity: string, case_positive_specimen_interval: int, case_onset_interval: int, process: string, exposure_yn: string, current_status: string, symptom_status: string, hosp_yn: string, icu_yn: string, death_yn: string, underlying_conditions_yn: string]

In [9]:
# Drop Columns: case_month, state_fips, county_fips, case positive specimen interval, case onset interval, process, exposure, current status, icu, death
#Ava edit: put ethnicity back in and dropped underlying_conditions and sympton_status
columns_to_drop = ['case_month', 'state_fips_code', 'county_fips_code','underlying_conditions_yn', "symptom_status", 'case_positive_specimen_interval', 'case_onset_interval', 'process', 'exposure_yn', 'current_status', 'icu_yn', 'death_yn']
covid_data_df_dropped_columns = covid_data_df_dropped.drop(*columns_to_drop)

In [10]:
covid_data_df_dropped_columns.show()

+---------+----------+--------------+------+-------+---------+-------+
|res_state|res_county|     age_group|   sex|   race|ethnicity|hosp_yn|
+---------+----------+--------------+------+-------+---------+-------+
|       OH|   BELMONT|            NA|    NA|     NA|       NA|Missing|
|       OH|    GALLIA|  0 - 17 years|    NA|     NA|       NA|Missing|
|       PA|       ELK|18 to 49 years|    NA|     NA|       NA|     No|
|       AL|TALLAPOOSA|       Missing|Female|Missing|  Missing|Missing|
|       IA|   CARROLL|  0 - 17 years|Female|     NA|       NA|Missing|
|       CO|    ELBERT|  0 - 17 years|Female|Missing|       NA|Missing|
|       SC|    JASPER|  0 - 17 years|Female|     NA|       NA|     No|
|       OH| JEFFERSON|  0 - 17 years|Female|     NA|       NA|     No|
|       MI|   LENAWEE|  0 - 17 years|Female|     NA|       NA|     No|
|       IL| ST. CLAIR|  0 - 17 years|Female|Missing|  Missing|Missing|
|       IL|STEPHENSON|  0 - 17 years|Female|     NA|       NA|     No|
|     

In [11]:
covid_data_df_dropped_columns_hospital = covid_data_df_dropped_columns.where(covid_data_df_dropped_columns.hosp_yn != 'Missing')

In [12]:
covid_data_df_dropped_columns_hospital.show()

+---------+----------+--------------+------+--------------+---------+-------+
|res_state|res_county|     age_group|   sex|          race|ethnicity|hosp_yn|
+---------+----------+--------------+------+--------------+---------+-------+
|       PA|       ELK|18 to 49 years|    NA|            NA|       NA|     No|
|       SC|    JASPER|  0 - 17 years|Female|            NA|       NA|     No|
|       OH| JEFFERSON|  0 - 17 years|Female|            NA|       NA|     No|
|       MI|   LENAWEE|  0 - 17 years|Female|            NA|       NA|     No|
|       IL|STEPHENSON|  0 - 17 years|Female|            NA|       NA|     No|
|       WA|    ASOTIN|18 to 49 years|Female|            NA|       NA|     No|
|       OH|  AUGLAIZE|18 to 49 years|Female|            NA|       NA|     No|
|       OH|  DEFIANCE|18 to 49 years|Female|            NA|       NA|     No|
|       MI|     EMMET|18 to 49 years|Female|            NA|       NA|     No|
|       OH|      PIKE|18 to 49 years|Female|            NA|     

In [13]:
covid_data_df_dropped_columns_hospital_again = covid_data_df_dropped_columns_hospital.where(covid_data_df_dropped_columns.hosp_yn != 'Unknown')

In [14]:
covid_data_df_dropped_columns_hospital_na = covid_data_df_dropped_columns_hospital_again.where(covid_data_df_dropped_columns_hospital_again.hosp_yn!='NA')
covid_data_df_dropped_columns_hospital_null = covid_data_df_dropped_columns_hospital_na.where(covid_data_df_dropped_columns_hospital_na.hosp_yn!='null')
covid_data_df_dropped_columns_hospital_null.show()

+---------+----------+--------------+------+--------------+---------------+-------+
|res_state|res_county|     age_group|   sex|          race|      ethnicity|hosp_yn|
+---------+----------+--------------+------+--------------+---------------+-------+
|       PA|       ELK|18 to 49 years|    NA|            NA|             NA|     No|
|       SC|    JASPER|  0 - 17 years|Female|            NA|             NA|     No|
|       OH| JEFFERSON|  0 - 17 years|Female|            NA|             NA|     No|
|       MI|   LENAWEE|  0 - 17 years|Female|            NA|             NA|     No|
|       IL|STEPHENSON|  0 - 17 years|Female|            NA|             NA|     No|
|       WA|    ASOTIN|18 to 49 years|Female|            NA|             NA|     No|
|       OH|  AUGLAIZE|18 to 49 years|Female|            NA|             NA|     No|
|       OH|  DEFIANCE|18 to 49 years|Female|            NA|             NA|     No|
|       MI|     EMMET|18 to 49 years|Female|            NA|             NA| 

In [ ]:
# those codes took away all hospitazation values werhe the condition was missing or unknown

In [ ]:
#removing every null value across all the columns    NA, Missing, Unknown, null

In [15]:
# remove missing values from resident state
res_state_df_unknown = covid_data_df_dropped_columns_hospital_null.where(covid_data_df_dropped_columns_hospital_null.res_state!= 'Unknown')
res_state_df_missing = res_state_df_unknown.where(res_state_df_unknown.res_state!= 'Missing')
res_state_df_na = res_state_df_missing.where(res_state_df_missing.res_state!='NA')
res_state_df_null = res_state_df_na.where(res_state_df_na.res_state!='null')

In [16]:
#remove missing values from state res_county
res_county_df_unknown = res_state_df_null.where(res_state_df_null.res_county!='Unknown')
res_county_df_missing = res_county_df_unknown.where(res_county_df_unknown.res_county!='Missing')
res_county_df_na = res_county_df_missing.where(res_county_df_missing.res_county!='NA')
res_county_df_null = res_county_df_na.where(res_county_df_na.res_county!='null')

In [17]:
#remove missing values from state age_group
age_group_df_unknown = res_county_df_null.where(res_county_df_null.age_group!='Unknown')
age_group_df_missing = age_group_df_unknown.where(age_group_df_unknown.age_group!='Missing')
age_group_df_na = age_group_df_missing.where(age_group_df_missing.age_group!='NA')
age_group_df_null = age_group_df_na.where(age_group_df_na.age_group!='null')

In [18]:
#removes missing values from sex
sex_df_unknown = age_group_df_null.where(age_group_df_null.sex!='Unknown')
sex_df_missing = sex_df_unknown.where(sex_df_unknown.sex!='Missing')
sex_df_na = sex_df_missing.where(sex_df_missing.sex!='NA')
sex_df_null = sex_df_na.where(sex_df_na.sex!='null')

In [19]:
#removes missing values from race
race_df_unknown = sex_df_null.where(sex_df_null.race!='Unknown')
race_df_missing = race_df_unknown.where(race_df_unknown.race!='Missing')
race_df_na = race_df_missing.where(race_df_missing.race!='NA')
race_df_null =race_df_na.where(race_df_na.race!='null')

In [ ]:
#removes missing values from symptom_status
#symptom_status_unknown = race_df_null.where(race_df_null.symptom_status!='Unknown')
#symptom_status_missing = symptom_status_unknown.where(symptom_status_unknown.symptom_status!='Missing')
#symptom_status_na = symptom_status_missing.where(symptom_status_missing.symptom_status!='NA')
#symptom_status_null = symptom_status_na.where(symptom_status_na.symptom_status!='null')

In [ ]:
#removes missing values from underlying_conditions_yn
#underlying_conditions_unknown = symptom_status_null.where(symptom_status_null.underlying_conditions_yn!='Unknown')
#underlying_conditions_missing = underlying_conditions_unknown.where(underlying_conditions_unknown.underlying_conditions_yn!='Missing')
#underlying_conditions_na = underlying_conditions_missing.where(underlying_conditions_missing.underlying_conditions_yn!='NA')
#underlying_conditions_null = underlying_conditions_na.where(underlying_conditions_na.underlying_conditions_yn!='null')

In [28]:
#removes missing values from ethnicity
ethnicity_df_unknown = race_df_null.where(race_df_null.ethnicity!='Unknown')
ethnicity_df_missing = ethnicity_df_unknown.where(ethnicity_df_unknown.ethnicity!='Missing')
ethnicity_df_na = ethnicity_df_missing.where(ethnicity_df_missing.ethnicity!='NA')
ethnicity_df_null =ethnicity_df_na.where(ethnicity_df_na.ethnicity!='null')

In [29]:
provisional_df = ethnicity_df_null.dropna()
provisional_df.show()

+---------+------------+--------------+------+--------------------+-------------------+-------+
|res_state|  res_county|     age_group|   sex|                race|          ethnicity|hosp_yn|
+---------+------------+--------------+------+--------------------+-------------------+-------+
|       NJ|      CAMDEN|18 to 49 years|Female|               Black|    Hispanic/Latino|     No|
|       NJ|      HUDSON|18 to 49 years|Female|               Black|    Hispanic/Latino|     No|
|       VA|NORFOLK CITY|18 to 49 years|Female|      Multiple/Other|    Hispanic/Latino|     No|
|       FL|    HERNANDO|  0 - 17 years|Female|               White|    Hispanic/Latino|     No|
|       OH|      BUTLER|18 to 49 years|Female|               White|    Hispanic/Latino|     No|
|       FL|      HENDRY|18 to 49 years|Female|               White|    Hispanic/Latino|     No|
|       FL|      MONROE|18 to 49 years|Female|               White|    Hispanic/Latino|     No|
|       FL|      DESOTO|  0 - 17 years| 

In [30]:
#created the provisional database 
#provisional_df is the provisional database
provisional_df.count()

6952336

In [ ]:
# Use get_dummies() to create variables for text features: all remaining columns

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://'insert url'"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write new df to table in RDS
insertnewdfname.write.jdbc(url=jdbc_url, table='insert new table name', mode=mode, properties=config)

In [ ]:
#Start machine learning code here